In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# === 1. 設定 (Config) ===
CONFIG = {
    "text_vocab_size": 1000,    # テキストの語彙数 (簡易版)
    "speech_vocab_size": 500,   # 音声トークン(HuBERT)の種類数
    "dim": 256,                 # 埋め込み次元
    "layers": 2,                # Transformer層数
    "heads": 4,                 # Attentionヘッド数
    "max_len": 50               # 生成する最大長
}

# === 2. モデル定義 (The Spirit LM) ===
class RunnableSpiritLM(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        # A. 埋め込み層 (Text & Speech)
        self.text_emb = nn.Embedding(cfg["text_vocab_size"], cfg["dim"])
        self.speech_emb = nn.Embedding(cfg["speech_vocab_size"], cfg["dim"])
        
        # B. 位置エンコーディング (簡易的)
        self.pos_emb = nn.Parameter(torch.zeros(1, 1024, cfg["dim"]))
        
        # C. Backbone (Llama 2相当のTransformer)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=cfg["dim"], 
            nhead=cfg["heads"], 
            dim_feedforward=cfg["dim"]*4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=cfg["layers"])
        
        # D. 出力層 (テキストと音声を結合した全語彙空間へ射影)
        # 出力次元 = テキスト語彙 + 音声語彙
        self.total_vocab = cfg["text_vocab_size"] + cfg["speech_vocab_size"]
        self.head = nn.Linear(cfg["dim"], self.total_vocab)

    def forward(self, input_ids, modality_mask):
        """
        input_ids: [Batch, Seq_Len] (トークンID)
        modality_mask: [Batch, Seq_Len] (0=Text, 1=Speech)
        """
        B, T = input_ids.shape
        
        # 1. モダリティに応じたEmbeddingの選択 (ここがSpirit LMの肝！)
        # maskが0ならtext_emb, 1ならspeech_embを使う
        # ※ input_idsはそれぞれの語彙サイズ内に収まっている前提
        
        # テキスト用埋め込み
        t_emb = self.text_emb(input_ids.clamp(0, self.cfg["text_vocab_size"]-1))
        # 音声用埋め込み
        s_emb = self.speech_emb(input_ids.clamp(0, self.cfg["speech_vocab_size"]-1))
        
        # マスクを使ってブレンド (ブロードキャスト)
        mask_expanded = modality_mask.unsqueeze(-1).float() # [B, T, 1]
        x = t_emb * (1 - mask_expanded) + s_emb * mask_expanded
        
        # 2. 位置情報を加算
        x = x + self.pos_emb[:, :T, :]
        
        # 3. Transformer処理
        x = self.transformer(x)
        
        # 4. 次トークン予測
        logits = self.head(x) # [B, T, Total_Vocab]
        return logits

# === 3. 生成ロジック (Generation Loop) ===
def generate(model, start_tokens, start_masks, max_new_tokens=20):
    model.eval()
    
    # 現在のシーケンス (List)
    curr_ids = start_tokens.tolist()[0]
    curr_masks = start_masks.tolist()[0]
    
    print(f"--- 生成開始 (初期入力長: {len(curr_ids)}) ---")
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            # テンソル化
            input_tensor = torch.tensor([curr_ids], dtype=torch.long)
            mask_tensor = torch.tensor([curr_masks], dtype=torch.long)
            
            # 推論
            logits = model(input_tensor, mask_tensor)
            
            # 最後のトークンのロジットを取得
            next_token_logits = logits[0, -1, :]
            
            # サンプリング (今回は簡易的にArgmax)
            next_id = torch.argmax(next_token_logits).item()
            
            # --- 判定: これはテキストか音声か？ ---
            # Spirit LMの出力層は [0 ~ text_vocab] がテキスト
            # [text_vocab ~ total_vocab] が音声 とマッピングされていると仮定
            
            if next_id < CONFIG["text_vocab_size"]:
                # テキストと判定
                modality = 0 
                token_type = "📝 Text"
                # トークンIDをそのまま表示
                display_val = f"ID:{next_id}"
            else:
                # 音声と判定
                modality = 1
                token_type = "🎵 Speech"
                # 音声IDに戻す (Global ID -> Local Speech ID)
                speech_id = next_id - CONFIG["text_vocab_size"]
                display_val = f"Unit:{speech_id}"
            
            print(f"生成: {token_type} | {display_val}")
            
            # シーケンスに追加（自己回帰）
            # 次の入力のために、Speechの場合はIDをローカルIDに直して格納するなど
            # 実装によって異なりますが、ここではGlobal IDのまま扱わないよう調整
            # ※簡易化のため、input_idsには常にLocal IDが入ると仮定して補正
            if modality == 1:
                input_id_for_next = next_id - CONFIG["text_vocab_size"]
            else:
                input_id_for_next = next_id
                
            curr_ids.append(input_id_for_next)
            curr_masks.append(modality)
            
    return curr_ids, curr_masks

# === 4. メイン実行ブロック ===

if __name__ == "__main__":
    # モデルのインスタンス化
    spirit_lm = RunnableSpiritLM(CONFIG)
    print("✅ モデル構築完了")

    # --- ダミー入力データの作成 ---
    # シチュエーション: "Hello" (Text) と言った後に 音声トークンが続くような状態を模擬
    # 0=Text, 1=Speech
    
    # Text: ID 10, 20 ("He", "llo")
    input_ids = [10, 20] 
    masks     = [ 0,  0] 
    
    # Speech: ID 5, 6, 7 (音声トークン) が混ざってくる (Interleaved)
    input_ids.extend([5, 6, 7])
    masks.extend    ([1, 1, 1])
    
    # Tensorに変換
    input_tensor = torch.tensor([input_ids], dtype=torch.long)
    mask_tensor = torch.tensor([masks], dtype=torch.long)
    
    print(f"初期入力: {input_ids}")
    print(f"初期マスク: {masks} (0=Text, 1=Speech)")
    
    # 生成実行
    print("\n--- Spirit LM 推論ループ ---")
    final_ids, final_masks = generate(spirit_lm, input_tensor, mask_tensor)
    
    print("\n✅ 生成終了")

✅ モデル構築完了
初期入力: [10, 20, 5, 6, 7]
初期マスク: [0, 0, 1, 1, 1] (0=Text, 1=Speech)

--- Spirit LM 推論ループ ---
--- 生成開始 (初期入力長: 5) ---
生成: 🎵 Speech | Unit:355
生成: 📝 Text | ID:254
生成: 🎵 Speech | Unit:283
生成: 🎵 Speech | Unit:126
生成: 📝 Text | ID:368
生成: 📝 Text | ID:572
生成: 🎵 Speech | Unit:230
生成: 🎵 Speech | Unit:80
生成: 🎵 Speech | Unit:208
生成: 📝 Text | ID:539
生成: 🎵 Speech | Unit:126
生成: 📝 Text | ID:368
生成: 📝 Text | ID:766
生成: 📝 Text | ID:928
生成: 📝 Text | ID:1
生成: 🎵 Speech | Unit:463
生成: 🎵 Speech | Unit:442
生成: 📝 Text | ID:607
生成: 🎵 Speech | Unit:286
生成: 🎵 Speech | Unit:364

✅ 生成終了
